# Combined Indicators

This notebook is meant to compile all of the indicators into a single dataframe for easy analysis.  

The steps required to do this are:  
* Gather data from source web sites  
* Clean data and choose important data points to put into a single .csv or database  
* Eliminate trends from data using Mayer's multiple and/or z-scores  